In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
import os

#@markdown ## 1. ComfyUI 설치 (구글드라이브 연동)
#@markdown ##### Install ComfyUI
Google_Install = True
#@markdown **지우고 재설치 (delete and reinstall)**
Reinstall = False #@param {type:"boolean"}
#@markdown <details>
#@markdown   <summary>설치된 커스텀 노드 자세히 펼쳐보기 (details)</summary>
#@markdown
#@markdown **WD14-Tagger** 이미지->프롬프트
#@markdown > https://github.com/pythongosssss/ComfyUI-WD14-Tagger 
#@markdown
#@markdown **MTB Nodes** 이미지 / 애니메이션관련
#@markdown > https://github.com/melMass/comfy_mtb
#@markdown
#@markdown **rgthree** 편의성
#@markdown > https://github.com/rgthree/rgthree-comfy
#@markdown
#@markdown **was-node-suite** 편의성
#@markdown > https://github.com/WASasquatch/was-node-suite-comfyui
#@markdown
#@markdown **ComfyUI_ADV_CLIP_emb** A1111 프롬프트 가중치
#@markdown > https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb
#@markdown
#@markdown **ComfyUI-Easy-Use** Dynamic Prompt (Wildcards)
#@markdown > https://github.com/yolain/ComfyUI-Easy-Use
#@markdown
#@markdown **comfyui-lama-remover** Lama (remove object)
#@markdown > https://github.com/Layer-norm/comfyui-lama-remover
#@markdown
#@markdown **ComfyUI_essentials** Image, Mask
#@markdown > https://github.com/cubiq/ComfyUI_essentials
#@markdown
#@markdown </details>

HomePath = '/content'
Workspace = 'install_comfyui'

if Google_Install:
  Workspace = 'drive/MyDrive/install_comfyui'
  HomePath = '/content/drive/MyDrive'
  from google.colab import drive
  drive.mount('/content/drive')
  if Reinstall:
    !rm -rf {HomePath}/{Workspace}

print('install...')
if not os.path.isdir(f'/content/{Workspace}'):
  with capture.capture_output() as cap:
    !wget https://huggingface.co/ninjaneural/webui/resolve/main/direct/install_comfyui_pack_20240516.tar -O install_comfyui.tar
    !tar -C {HomePath} -xvf install_comfyui.tar
    !rm install_comfyui.tar
    !wget https://raw.githubusercontent.com/ninjaneural/comfyui/master/misc/extra_model_paths.yaml -O /content/{Workspace}/extra_model_paths.yaml
    !sed -i -e "/    init_custom_nodes()/a\    folder_paths.add_model_folder_path('AnimateDiffEvolved_Models', '/content/animatediff/models')\n    folder_paths.add_model_folder_path('AnimateDiffMotion_LoRA', '/content/animatediff/motion_lora')" /content/{Workspace}/main.py
    %cd /content/{Workspace}

print("설치완료. 실행환경 설치을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 2. 실행환경 설치
#@markdown ##### Install Env

%cd /content

!mkdir /content/models
!mkdir /content/models/checkpoints
!mkdir /content/models/controlnet_models
!mkdir /content/models/upscale_models
!mkdir /content/models/loras
!mkdir /content/models/vae
!mkdir /content/models/clip_vision
!mkdir /content/models/clip

print('install library...')
with capture.capture_output() as cap:
  !pip install kornia
  !pip install torchsde
  !pip install insightface
  !pip install timm

  !pip install diffusers==0.25.0
  !pip install accelerate
  !pip install gradio==4.24.0
  !pip install av
  !pip install einops==0.7.0
  !pip install onnxruntime
  !pip install omegaconf>=2.0.0
  !pip install fvcore

  !pip install segment_anything
  !pip install addict
  !pip install yapf

print("설치완료. 모델 다운로드를 진행해주세요")


In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 3. 모델 다운로드
#@markdown ##### Model Download

#@markdown **모델(checkpoint) URL (colab에 다운받는 모델)**
Checkpoint_Url = 'https://huggingface.co/Meina/MeinaPastel/resolve/main/MeinaPastelV5%20-%20Baked%20VAE.safetensors' #@param {type:"string"}
#@markdown **모델 파일명**
Checkpoint_Filename = 'MeinaPastelV5Baked.safetensors' #@param {type:"string"}
#@markdown **모델 다운로드**
Checkpoint_Download = True #@param {type:"boolean"}

#@markdown -----
#@markdown *컨트롤넷 모델*

#@markdown **ControlNet 기본 모델 다운로드**
#@markdown > *openpose, lineart, softedge, depth, canny, tile, ip2p*  
ControlNet = True #@param {type:"boolean"}

#@markdown **Sai 모델 다운로드**
#@markdown > *canny, depth, recolor, sketch*  
ControlNet_SAI = False #@param {type:"boolean"}

#@markdown **Kohya 모델 다운로드**
#@markdown > *canny_anime, depth_anime, openpose_anime, scribble_anime*  
ControlNet_KOHYA = False #@param {type:"boolean"}

#@markdown **T2I Adapter 모델 다운로드**
#@markdown > *canny, openpose, sketch*  
ControlNet_T2I = False #@param {type:"boolean"}

#@markdown -----
#@markdown *커스텀 노드 모델*

#@markdown **AnimateDiff 모델 다운로드**
AnimateDiff_Evolved = True #@param {type:"boolean"}

#@markdown **IPAdapter plus 모델 다운로드**
IPAdapter_plus = False #@param {type:"boolean"}

#@markdown **SVD 모델 다운로드**
SVD_Checkpoint = False #@param {type:"boolean"}

#@markdown **insightface 설치 (reactor 노드사용)**
Insightface = False #@param {type:"boolean"}

print('clip download...')
with capture.capture_output() as cap:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/models/clip_vision -o model-ViT-H.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -d /content/models/clip_vision -o model-ViT-bigG.safetensors
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -d /content/models/clip_vision/ -o clip_vit14.bin
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/resolve/main/open_clip_pytorch_model.bin -d /content/models/clip -o open_clip_pytorch_model.bin

print('controlnet download...')
with capture.capture_output() as cap:

  ControlNet_Model_Path = "/content/models/controlnet_models"
  if ControlNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors -d {ControlNet_Model_Path} -o diffusers_xl_canny_mid.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors -d {ControlNet_Model_Path} -o diffusers_xl_depth_mid.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors -d {ControlNet_Model_Path} -o sargezt_xl_softedge.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors -d {ControlNet_Model_Path} -o thibaud_xl_openpose.safetensors

  if ControlNet_SAI:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_256lora.safetensors -d {ControlNet_Model_Path} -o sai_xl_canny_256lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_256lora.safetensors -d {ControlNet_Model_Path} -o sai_xl_depth_256lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors -d {ControlNet_Model_Path} -o sai_xl_recolor_256lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors -d {ControlNet_Model_Path} -o sai_xl_sketch_256lora.safetensors

  if ControlNet_KOHYA:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors -d {ControlNet_Model_Path} -o kohya_controllllite_xl_canny_anime.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors -d {ControlNet_Model_Path} -o kohya_controllllite_xl_depth_anime.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors -d {ControlNet_Model_Path} -o kohya_controllllite_xl_openpose_anime_v2.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors -d {ControlNet_Model_Path} -o kohya_controllllite_xl_scribble_anime.safetensors

  if ControlNet_T2I:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors -d {ControlNet_Model_Path} -o t2i-adapter_xl_canny.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors -d {ControlNet_Model_Path} -o t2i-adapter_xl_openpose.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors -d {ControlNet_Model_Path} -o t2i-adapter_xl_sketch.safetensors

print('animatediff download...')
with capture.capture_output() as cap:

  if AnimateDiff_Evolved:
    !mkdir /content/animatediff
    !mkdir /content/animatediff/models
    !mkdir /content/animatediff/motion_lora
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sdxl_v10_beta.ckpt -d /content/animatediff/models -o mm_sdxl_v10_beta.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hotshotco/Hotshot-XL/resolve/main/hsxl_temporal_layers.safetensors -d /content/animatediff/models -o hsxl_temporal_layers.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.ckpt -d /content/animatediff/models -o temporaldiff-v1-animatediff.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt -d /content/animatediff/motion_lora -o v2_lora_PanLeft.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt -d /content/animatediff/motion_lora -o v2_lora_PanRight.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt -d /content/animatediff/motion_lora -o v2_lora_ZoomOut.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt -d /content/animatediff/motion_lora -o v2_lora_ZoomIn.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt -d /content/animatediff/motion_lora -o v2_lora_TiltUp.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt -d /content/animatediff/motion_lora -o v2_lora_TiltDown.ckpt
    
print('ipadapter download...')
with capture.capture_output() as cap:

  if IPAdapter_plus:
    !mkdir /content/ipadapter

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -d /content/ipadapter -o ip-adapter_sdxl_vit-h.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -d /content/ipadapter -o ip-adapter-plus_sdxl_vit-h.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -d /content/ipadapter -o ip-adapter-plus-face_sdxl_vit-h.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin -d /content/ipadapter -o ip-adapter-faceid_sdxl.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin -d /content/ipadapter -o ip-adapter-faceid-plusv2_sdxl.bin

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors -d /content/models/loras -o ip-adapter-faceid_sdxl_lora.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors -d /content/models/loras -o ip-adapter-plusv2_sdxl_lora.safetensors
    
# svd checkpoint
print('svd download...')
with capture.capture_output() as cap:
  if SVD_Checkpoint:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid/resolve/main/svd.safetensors -d /content/models/checkpoints -o svd.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors -d /content/models/checkpoints -o svd_xt.safetensors

  if Insightface:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -d /content/{Workspace}/models/facerestore_models -o GFPGANv1.4.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ninjaneural/webui/resolve/main/inswapper_128.onnx -d /content/{Workspace}/models/insightface -o inswapper_128.onnx
    !pip install insightface

# checkpoint
print('checkpoint download...')
with capture.capture_output() as cap:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d /content/models/checkpoints -o {Checkpoint_Filename}

  # upscaler
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/models/upscale_models -o 4x-UltraSharp.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gemasai/4x_NMKD-Siax_200k/resolve/main/4x_NMKD-Siax_200k.pth -d /content/models/upscale_models -o 4x_NMKD-Siax_200k.pth

  # vae
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/models/vae -o vae-ft-mse-840000-ema-pruned.vae.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/models/vae -o kl-f8-anime2.ckpt

  # lora
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/models/loras -o add_detail.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors" -d /content/models/loras -o lcm-lora-sdxl.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_2step_lora.safetensors" -d /content/models/loras -o sdxl_lightning_2step_lora.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_4step_lora.safetensors" -d /content/models/loras -o sdxl_lightning_4step_lora.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_8step_lora.safetensors" -d /content/models/loras -o sdxl_lightning_8step_lora.safetensors


%cd /content/{Workspace}

print("다운로드완료. ComfyUI 실행을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## (옵션) 추가 커스텀 노드 설치
#@markdown ##### (Optional) Install Custom Nodes
#@markdown * Gemini API키
GEMINI_API_KEY = '' #@param {type:"string"}

#@markdown * ReActor
ReActor = False #@param {type:"boolean"}

#@markdown * DreamTalk
DREAMTALK = False #@param {type:"boolean"}

#@markdown * IDM-VTON
IDM_VTON = True #@param {type:"boolean"}

#@markdown <br/>
#@markdown <details>
#@markdown   <summary>추가 커스텀노드 자세히 펼쳐보기 (details)</summary>
#@markdown
#@markdown **comfyui_segment_anything** Segment Anything, DINO
#@markdown > https://github.com/storyicon/comfyui_segment_anything
#@markdown
#@markdown **ComfyUI-Video-Matting** 배경 분리 Robust Video Matting (RVM)
#@markdown > https://github.com/Fannovel16/ComfyUI-Video-Matting
#@markdown
#@markdown **ComfyUI-Gemini** Gemini 연동
#@markdown > https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini
#@markdown
#@markdown </details>

%cd /content/{Workspace}/custom_nodes

print('custom nodes download...')
with capture.capture_output() as cap:
  !git clone https://github.com/storyicon/comfyui_segment_anything
  !git clone https://github.com/Fannovel16/ComfyUI-Video-Matting

  if DREAMTALK:
    !git clone https://github.com/neuralninja22/ComfyUI_DDreamtalk
    %cd ComfyUI_DDreamtalk
    !pip install -r requirements.txt
    %cd ..
  
  if IDM_VTON:
    !git clone https://github.com/neuralninja22/IDM-VTON
    !apt install aria2

    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/openpose/ckpts/body_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/levihsu/OOTDiffusion/resolve/main/checkpoints/openpose/ckpts/body_pose_model.pth -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/openpose/ckpts -o body_pose_model.pth
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing/parsing_atr.onnx
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/humanparsing/parsing_atr.onnx -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing -o parsing_atr.onnx
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing/parsing_lip.onnx
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/humanparsing/parsing_lip.onnx -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing -o parsing_lip.onnx
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/densepose/model_final_162be9.pkl
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/densepose/model_final_162be9.pkl -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/densepose -o model_final_162be9.pkl
    
  if not ReActor:
    !mv comfyui-reactor-node ../../

  if GEMINI_API_KEY:
    !git clone https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini
    !sed -i -e 's/"api_key": "[^"]*"/"api_key": "{GEMINI_API_KEY}"/' ./ComfyUI-Gemini/config.json

%cd /content/{Workspace}

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## (옵션) SDXL 추가 커스텀 노드 설치
#@markdown ##### (Optional) Install SDXL Custom Nodes

#@markdown **comfyui-inpaint-nodes** Fooocus inpaint
#@markdown > https://github.com/Acly/comfyui-inpaint-nodes

%cd /content/{Workspace}/custom_nodes

print('custom nodes download...')
with capture.capture_output() as cap:
  !git clone https://github.com/Acly/comfyui-inpaint-nodes
  
%cd /content/{Workspace}

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown **ComfyUI-Gemini** Gemini 연동
#@markdown > https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini

GEMINI_API_KEY = '' #@param {type:"string"}

%cd /content/{Workspace}/custom_nodes

print('custom nodes download...')
with capture.capture_output() as cap:
  !git clone https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini
  !sed -i -e 's/"your key"/{GEMINI_API_KEY}/' ./ComfyUI-Gemini/config.json
    

%cd /content/{Workspace}

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 4. ComfyUI 실행
#@markdown ##### Run ComfyUI

#@markdown -----
#@markdown *터널링*

#@markdown **Localtunnel**
LocalTunnel = True #@param {type:"boolean"}

%cd /content/{Workspace}

if LocalTunnel:
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()

    print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')

  threading.Thread(target=iframe_thread, daemon=True, args=(18188,)).start()

!python main.py --port 18188 --dont-print-server